In [1]:
from gpseer.predictor import Predictor

In [2]:
from seqspace.gpm import GenotypePhenotypeMap
from epistasis.models import LinearEpistasisRegression

In [3]:
space = GenotypePhenotypeMap.from_json("linear.json")
predictor = Predictor(space, LinearEpistasisRegression, order=4)

In [4]:
predictor.sample_ref(100, "0000")

In [5]:
predictor.predictions.ref0000.samples.shape

(100, 16)

In [6]:
import numpy as np

In [7]:
np.mean(predictor.predictions.ref0000.samples, axis=0)

array([ 0.60662538,  0.87480932,  0.19046038,  0.53485519,  0.6695556 ,
        0.9295401 ,  0.88489032,  0.32511142,  0.24202871,  0.65163714,
        0.84728509,  0.39993656,  0.80565661,  0.96662956,  0.84929597,
        0.9817434 ], dtype=float32)

In [8]:
predictor.sample_to_convergence("0000",rtol=1e-1)

In [10]:
predictor.predictions.ref0000.samples.shape

(6150, 16)

In [14]:
2+2

4

In [20]:
t

NameError: name 't' is not defined